In [1]:
from LorisBallsBasedModel import *
import tensorflow as tf
import keras_tuner as kt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']
cars = pd.read_csv(r'data\car_evaluation_dataset\car_evaluation.csv', names=cols, header=None)
cars.head(3)

labels = pd.get_dummies(cars.output, prefix='condition')

X = cars[['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']]
y = labels.values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=42)

X_train_dict = X_train.to_dict(orient='list')
X_train_dict = {k: np.expand_dims(v, axis=-1) for k, v in X_train_dict.items()}
X_test_dict = X_test.to_dict(orient='list')
X_test_dict = {k: np.expand_dims(v, axis=-1) for k, v in X_test_dict.items()}

# SingleLayerPerceptron base hypermodel

In [4]:
def build_SLP(hp):
    embedding_dim = hp.Choice('features_embedding_dim',
                              [1, 3, 6])
    X_train_embedding_layers = {}
    for input_name in X_train.columns:
        X_train_embedding_layers[input_name] = StringEmbedding(X_train[input_name].unique(),
                                                               embedding_dim)

    processing_layer = InputsProcessing(X_train_embedding_layers)

    SLP = SingleLayerPerceptron(tf.keras.layers.Dense(4,
                                                      activation='softmax'),
                                processing_layer=processing_layer)
    SLP.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate',
                                                                           [.01, .005, .001, .0005, .0001])),
                metrics=['acc'])
    return SLP

# Cross validation random search

In [5]:
hyper_model_CV = HyperModelCrossValidation(build_model=build_SLP,
                                           nbr_folds=5,
                                           batch_sizes_to_try_list=[8, 16])

tuner = kt.RandomSearch(
    hypermodel=hyper_model_CV,
    objective="val_loss",
    max_trials=15,
    executions_per_trial=1,
    overwrite=True,
    directory="keras_tuners_logs",
    project_name="test_HyperModelCrossValidation",
)  # set 'overwrite' to true to resume a previous search

tuner.search(X_train_dict,
             y_train,
             epochs=10)

Trial 15 Complete [00h 00m 17s]
val_loss: 0.6406977415084839

Best val_loss So Far: 0.17156047821044923
Total elapsed time: 00h 05m 19s
INFO:tensorflow:Oracle triggered exit


In [6]:
tuner.results_summary(1)

Results summary
Results in keras_tuners_logs\test_HyperModelCrossValidation
Showing 1 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
features_embedding_dim: 3
learning_rate: 0.01
batch_size: 16
Score: 0.17156047821044923


In [7]:
train_tensor = tf.data.Dataset.from_tensor_slices((X_train_dict,
                                                   y_train))
train_tensor = train_tensor.cache().shuffle(300).batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_tensor = tf.data.Dataset.from_tensor_slices((X_test_dict,
                                                  y_test)).batch(8)

best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
SLP = build_SLP(best_hyperparameters)
history = SLP.fit(train_tensor,
                  epochs=10,
                  verbose=1)

Epoch 1/10
173/173 [==============================] - 1s 2ms/step - loss: 0.5429 - acc: 0.7988
Epoch 2/10
173/173 [==============================] - 0s 2ms/step - loss: 0.2482 - acc: 0.8907
Epoch 3/10
173/173 [==============================] - 0s 2ms/step - loss: 0.2047 - acc: 0.9204
Epoch 4/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1865 - acc: 0.9175
Epoch 5/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1720 - acc: 0.9349
Epoch 6/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1677 - acc: 0.9305
Epoch 7/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1666 - acc: 0.9313
Epoch 8/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1665 - acc: 0.9284
Epoch 9/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1622 - acc: 0.9356
Epoch 10/10
173/173 [==============================] - 0s 2ms/step - loss: 0.1605 - acc: 0.9356


In [8]:
SLP.evaluate(test_tensor)

44/44 [==============================] - 0s 2ms/step - loss: 0.1654 - acc: 0.9191


[0.16538073122501373, 0.9190751314163208]

# TODO: Use TensorBoard